In [1]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from textblob import Word
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
#get data
data = pd.read_csv('data\\text_emotion.csv',index_col='tweet_id')
data

,sentiment,author,status
tweet_id,,,
1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
...,...,...,...
1753918954,neutral,showMe_Heaven,@JohnLloydTaylor
1753919001,love,drapeaux,Happy Mothers Day All my love
1753919005,love,JenniRox,Happy Mother's Day to all the mommies out ther...


In [3]:
#Preprocessing
data=data.drop('author',axis=1)
#data=data.drop('tweet_id',axis=1)

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Shuvo
[nltk_data]     Podder\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Shuvo
[nltk_data]     Podder\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
#lowercase
lower_case = lambda x:" ".join(x.lower() for x in x.split())
data['status'] = data['status'].apply(lower_case)

#remove punctuation,symbols
data['status'] = data['status'].str.replace('[^\w\s]',' ')

#remove stopwords
stop_word = stopwords.words('english')
stop_words = lambda x:" ".join(x for x in x.split() if x not in stop_word)
data['status'] = data['status'].apply(stop_words)

#lematisation 
ls = lambda x:" ".join([Word(word).lemmatize() for word in x.split()])
data['status'] = data['status'].apply(ls)

#fixing repeatations
def repeat(text):
    pt=re.compile(r"(.)\1{2,}")
    return pt.sub(r"\1\1",text)
rt=lambda x:" ".join(repeat(x) for x in x.split())
data['status'] = data['status'].apply(rt)
data


,sentiment,status
tweet_id,,
1956967341,empty,tiffanylue know listenin bad habit earlier sta...
1956967666,sadness,layin n bed headache ughh waitin call
1956967696,sadness,funeral ceremony gloomy friday
1956967789,enthusiasm,want hang friend soon
1956968416,neutral,dannycastillo want trade someone houston ticke...
...,...,...
1753918954,neutral,johnlloydtaylor
1753919001,love,happy mother day love
1753919005,love,happy mother day mommy woman man long momma so...


In [7]:
freq= pd.Series(' '.join(data['status']).split()).value_counts()[-10000:]
#remove rarely words
freq=list(freq.index)
data['status'] = data['status'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data

,sentiment,status
tweet_id,,
1956967341,empty,know listenin bad habit earlier started freaki...
1956967666,sadness,layin n bed headache ughh waitin call
1956967696,sadness,funeral ceremony gloomy friday
1956967789,enthusiasm,want hang friend soon
1956968416,neutral,want trade someone houston ticket one
...,...,...
1753918954,neutral,johnlloydtaylor
1753919001,love,happy mother day love
1753919005,love,happy mother day mommy woman man long momma so...


In [8]:
data.to_csv("data\\clean_data.csv")

In [9]:
data=pd.read_csv("data\\clean_data.csv")
data

,tweet_id,sentiment,status
0,1956967341,empty,know listenin bad habit earlier started freaki...
1,1956967666,sadness,layin n bed headache ughh waitin call
2,1956967696,sadness,funeral ceremony gloomy friday
3,1956967789,enthusiasm,want hang friend soon
4,1956968416,neutral,want trade someone houston ticket one
...,...,...,...
39995,1753918954,neutral,johnlloydtaylor
39996,1753919001,love,happy mother day love
39997,1753919005,love,happy mother day mommy woman man long momma so...
39998,1753919043,happiness,niariley wassup beautiful follow peep new hit ...


In [10]:
# Dropping rows with other emotion labels
#data = data.drop(data[data.sentiment == 'love'].index)
#data = data.drop(data[data.sentiment == 'boredom'].index)
#data = data.drop(data[data.sentiment == 'fun'].index)
#data = data.drop(data[data.sentiment == 'enthusiasm'].index)
data = data.drop(data[data.sentiment == 'anger'].index)
data = data.drop(data[data.sentiment == 'empty'].index)
data = data.drop(data[data.sentiment == 'relief'].index)
data = data.drop(data[data.sentiment == 'surprise'].index)
data = data.drop(data[data.sentiment == 'hate'].index)
data = data.drop(data[data.sentiment == 'neutral'].index)
data = data.drop(data[data.sentiment == 'worry'].index)
data

,tweet_id,sentiment,status
1,1956967666,sadness,layin n bed headache ughh waitin call
2,1956967696,sadness,funeral ceremony gloomy friday
3,1956967789,enthusiasm,want hang friend soon
6,1956968487,sadness,sleep im thinking old friend want married damn...
8,1956969035,sadness,charlene love miss
...,...,...,...
39994,1753918900,happiness,succesfully following tayla
39996,1753919001,love,happy mother day love
39997,1753919005,love,happy mother day mommy woman man long momma so...
39998,1753919043,happiness,niariley wassup beautiful follow peep new hit ...


In [11]:
#Encoding output labels
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.sentiment.values.astype('U'))

#split data into train 90% and test 10%
x_train,x_val,y_train,y_val = train_test_split(data.status.values.astype('U'),
                                              y,stratify=y,random_state = 42,
                                              test_size = 0.1, shuffle = True)

In [12]:
# Extracting TF-IDF parameters
tfidf = TfidfVectorizer(max_features = 1000,analyzer ='word',ngram_range=(1,3))
x_train_tfidf = tfidf.fit_transform(x_train)
x_val_tfidf =tfidf.fit_transform(x_val)

# Extracting Count Vectors Parameters
count_vect = CountVectorizer(analyzer='word')
count_vect.fit(data['status'].values.astype('U'))
x_train_count = count_vect.transform(x_train)
x_val_count = count_vect.transform(x_val)

In [13]:
#Naive Bayes Classifier
nb = MultinomialNB()
nb.fit(x_train_count,y_train)
y_pred = nb.predict(x_val_count)
#Evaluate
print('Accuracy %s' % accuracy_score(y_pred, y_val))


Accuracy 0.5363260484347312


In [14]:
import pickle
pickle.dump(nb,open('data\\model.sav','wb'))

In [15]:
model  = pickle.load(open('data\\model.sav','rb'))

In [19]:
# Read songs sentiment
song =pd.read_csv("data\\song_lyrics.csv")
song

,sentiment,song
0,sadness,I am
1,happiness,you are
2,sadness,I jshd sd
3,happiness,I am gshssj


In [17]:
def emotion(status):
    text = pd.DataFrame([status])
    
    # Doing some preprocessing on these text as done before
    text[0] = text[0].str.replace('[^\w\s]',' ')
    from nltk.corpus import stopwords
    stop = stopwords.words('english')
    text[0] = text[0].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    from textblob import Word
    text[0] = text[0].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    # Extracting Count Vectors feature from our text
    text_count = count_vect.transform(text[0])
    #Predicting the emotion of the tweet using our already trained linear SVM
    result = model.predict(text_count)
    #print(result)
    #lbl_enc.transform(data.sentiment.values.astype('U'))
    #print(lbl_enc.inverse_transform([result]))
    emotions= lbl_enc.inverse_transform([result])
    for e in emotions:
        st=e
    print("You seems %s today. Here is your recomended songs list: " %st)
    print(song.loc[song['sentiment']==st])
    

### User Input Though text

In [18]:
str = input()
emotion(str)

i am very happy today
You seems happiness today. Here is your recomended songs list: 
   sentiment     song
1  happiness  you are


C:\Users\Shuvo Podder\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### User Input Though Speech

In [21]:
import speech_recognition as sr

r=sr.Recognizer()
with sr.Microphone() as source:
    print('Say something')
    audio=r.listen(source)   #audio_data = r.record(source, duration=5)
    print("................................")
    
try:
    str=r.recognize_google(audio)
    print("Text: "+str);
    emotion(str)
    
except:
    print('error')
    pass
    

Say something
................................
Text: I am very happy today. Lucky
You seems happiness today. Here is your recomended songs list: 
   sentiment     song
1  happiness  you are


C:\Users\Shuvo Podder\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
